In [3]:
import numpy as np
from scipy import interpolate
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

# Initialize Dash app
app = dash.Dash(__name__)

# Default grid size
default_grid_size = 10

# Create a function to generate meshgrid and calculate Z values based on a formula
def generate_mesh_and_z(grid_size, time_factor, formula):
    x = np.linspace(-5, 5, grid_size)  # X coordinates
    y = np.linspace(-5, 5, grid_size)  # Y coordinates
    X, Y = np.meshgrid(x, y)  # Create a meshgrid for X and Y

    # Define the formula using numpy
    Z = eval(formula)  # Evaluate the formula
    return X, Y, Z

# Function to create B-spline lines
def create_bspline_lines(X, Y, Z):
    """Create B-spline lines for the grid."""
    lines = []
    grid_size = X.shape[0]  # Get the current grid size from X
    for i in range(grid_size):
        # Define control points for horizontal B-spline curves
        tck_h, _ = interpolate.splprep([X[i, :].flatten(), Y[i, :].flatten(), Z[i, :].flatten()], k=3, s=0)
        u_h = np.linspace(0, 1, num=100)
        out_h = interpolate.splev(u_h, tck_h)

        # Add B-spline curve to the lines
        lines.append(go.Scatter3d(
            x=out_h[0],
            y=out_h[1],
            z=out_h[2],
            mode='lines',
            line=dict(color='blue', width=2),
            name=f'B-spline Line {i}'
        ))

    for j in range(grid_size):
        # Define control points for vertical B-spline curves
        tck_v, _ = interpolate.splprep([X[:, j].flatten(), Y[:, j].flatten(), Z[:, j].flatten()], k=3, s=0)
        u_v = np.linspace(0, 1, num=100)
        out_v = interpolate.splev(u_v, tck_v)

        # Add B-spline curve to the lines
        lines.append(go.Scatter3d(
            x=out_v[0],
            y=out_v[1],
            z=out_v[2],
            mode='lines',
            line=dict(color='blue', width=2),
            name=f'B-spline Line {j + grid_size}'
        ))

    return lines

# Layout of the Dash app
app.layout = html.Div([
    dcc.Graph(id='3d-surface',config={'scrollZoom': True},style={'height': '80vh', 'width': '100%'}),
    html.Button('Start Animation', id='start-button', n_clicks=0),
    html.Button('Stop Animation', id='stop-button', n_clicks=0),
    dcc.Interval(
        id='interval-component',
        interval=100,  # in milliseconds
        n_intervals=0,
        disabled=True  # Start disabled
    ),
    dcc.Store(id='current-time-factor', data=0),  # Store the current time factor
    dcc.Store(id='camera-settings', data={}),  # Store camera settings

    # Sliders for camera control
    html.Div([
        html.Label('Camera Zoom'),
        dcc.Slider(
            id='zoom-slider',
            min=1,
            max=10,
            value=3,
            marks={i: str(i) for i in range(1, 11)},
            step=0.1
        ),
        html.Label('Camera Rotation (X-axis)'),
        dcc.Slider(
            id='rotation-x-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Camera Rotation (Y-axis)'),
        dcc.Slider(
            id='rotation-y-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Camera Rotation (Z-axis)'),
        dcc.Slider(
            id='rotation-z-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Grid Size'),
        dcc.Slider(
            id='grid-size-slider',
            min=2,
            max=100,  # Set maximum grid size to 100
            value=default_grid_size,
            marks={i: str(i) for i in range(2, 101, 10)},
            step=1
        )
    ]),

    # Input for custom formula
    html.Div([
        html.Label('Custom Formula for Surface (e.g., np.sin(np.sqrt(X**2 + Y**2) + time_factor)'),
        dcc.Input(id='formula-input', type='text', value='np.sin(np.sqrt(X**2 + Y**2) + time_factor)', style={'width': '100%'}),
        html.Button('Submit Formula', id='submit-formula-button', n_clicks=0)
    ]),

    # Toggle buttons for surface and grid visibility
    html.Div([
        html.Button('Toggle Surface', id='toggle-surface-button', n_clicks=0),
        html.Button('Toggle Grid', id='toggle-grid-button', n_clicks=0)
    ])
])

# Callback to update the graph
@app.callback(
    Output('3d-surface', 'figure'),
    Input('interval-component', 'n_intervals'),
    Input('current-time-factor', 'data'),
    Input('camera-settings', 'data'),
    Input('zoom-slider', 'value'),
    Input('rotation-x-slider', 'value'),
    Input('rotation-y-slider', 'value'),
    Input('rotation-z-slider', 'value'),
    Input('grid-size-slider', 'value'),
    Input('formula-input', 'value'),
    Input('toggle-surface-button', 'n_clicks'),
    Input('toggle-grid-button', 'n_clicks')
)
def update_graph(n, current_time_factor, camera_settings, zoom, rotation_x, rotation_y, rotation_z, grid_size, formula, surface_clicks, grid_clicks):
    # Generate new mesh and Z values based on the current grid size and formula
    X, Y, Z = generate_mesh_and_z(grid_size, current_time_factor + n * 0.1, formula)

    # Create the figure
    fig = go.Figure()

    # Add mesh surface
    fig.add_trace(go.Surface(
        z=Z,
        x=X,
        y=Y,
        colorscale='Viridis',
        opacity=0.8,
        name='Mesh Surface'
    ))

    # Add B-spline lines
    lines = create_bspline_lines(X, Y, Z)
    for line in lines:
        fig.add_trace(line)

    # Calculate camera position based on rotations
    eye_x = zoom * np.sin(np.radians(rotation_y)) * np.cos(np.radians(rotation_x))
    eye_y = zoom * np.sin(np.radians(rotation_x)) * np.sin(np.radians(rotation_y))
    eye_z = zoom * np.cos(np.radians(rotation_x))

    # Set layout with camera settings
    fig.update_layout(
        title='Nurbs Surface',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(
                eye=dict(x=eye_x, y=eye_y, z=eye_z),  # Updated based on zoom and rotations
                up=dict(x=0, y=0, z=1)
            )
        ),
        showlegend=False
    )

    # Toggle visibility based on button clicks
    if surface_clicks % 2 == 1:
        fig.data[0].visible = not fig.data[0].visible  # Toggle surface visibility

    if grid_clicks % 2 == 1:
        for i in range(1, len(fig.data)):
            fig.data[i].visible = not fig.data[i].visible  # Toggle grid line visibility

    return fig

# Callback to start and stop the animation
@app.callback(
    Output('interval-component', 'disabled'),
    Output('current-time-factor', 'data'),
    Input('start-button', 'n_clicks'),
    Input('stop-button', 'n_clicks'),
    prevent_initial_call=True
)
def toggle_animation(start_clicks, stop_clicks):
    ctx = dash.callback_context

    if ctx.triggered:
        # Determine which button was clicked
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'start-button':
            # Start the animation
            return False, dash.no_update  # Enable the interval

        elif button_id == 'stop-button':
            # Stop the animation
            return True, dash.no_update  # Disable the interval

    return True, dash.no_update  # Default to disabled

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)

In [7]:
import numpy as np
from scipy import interpolate
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go

# Initialize Dash app
app = dash.Dash(__name__)

# Default grid size
default_grid_size = 10

# Create a function to generate meshgrid and calculate Z values based on a formula
def generate_mesh_and_z(grid_size, time_factor, formula):
    x = np.linspace(-5, 5, grid_size)  # X coordinates
    y = np.linspace(-5, 5, grid_size)  # Y coordinates
    X, Y = np.meshgrid(x, y)  # Create a meshgrid for X and Y

    # Define the formula using numpy
    Z = eval(formula)  # Evaluate the formula
    return X, Y, Z

# Function to create B-spline lines
def create_bspline_lines(X, Y, Z):
    """Create B-spline lines for the grid."""
    lines = []
    grid_size = X.shape[0]  # Get the current grid size from X
    for i in range(grid_size):
        # Define control points for horizontal B-spline curves
        tck_h, _ = interpolate.splprep([X[i, :].flatten(), Y[i, :].flatten(), Z[i, :].flatten()], k=3, s=0)
        u_h = np.linspace(0, 1, num=100)
        out_h = interpolate.splev(u_h, tck_h)

        # Add B-spline curve to the lines
        lines.append(go.Scatter3d(
            x=out_h[0],
            y=out_h[1],
            z=out_h[2],
            mode='lines',
            line=dict(color='blue', width=2),
            name=f'B-spline Line {i}'
        ))

    for j in range(grid_size):
        # Define control points for vertical B-spline curves
        tck_v, _ = interpolate.splprep([X[:, j].flatten(), Y[:, j].flatten(), Z[:, j].flatten()], k=3, s=0)
        u_v = np.linspace(0, 1, num=100)
        out_v = interpolate.splev(u_v, tck_v)

        # Add B-spline curve to the lines
        lines.append(go.Scatter3d(
            x=out_v[0],
            y=out_v[1],
            z=out_v[2],
            mode='lines',
            line=dict(color='blue', width=2),
            name=f'B-spline Line {j + grid_size}'
        ))

    return lines

# Layout of the Dash app
app.layout = html.Div([
    dcc.Graph(id='3d-surface',config={'scrollZoom': True},style={'height': '80vh', 'width': '100%'}),
    html.Button('Start Animation', id='start-button', n_clicks=0),
    html.Button('Stop Animation', id='stop-button', n_clicks=0),
    dcc.Interval(
        id='interval-component',
        interval=100,  # in milliseconds
        n_intervals=0,
        disabled=True  # Start disabled
    ),
    dcc.Store(id='current-time-factor', data=0),  # Store the current time factor
    dcc.Store(id='camera-settings', data={}),  # Store camera settings

    # Sliders for camera control
    html.Div([
        html.Label('Camera Zoom'),
        dcc.Slider(
            id='zoom-slider',
            min=1,
            max=10,
            value=3,
            marks={i: str(i) for i in range(1, 11)},
            step=0.1
        ),
        html.Label('Camera Rotation (X-axis)'),
        dcc.Slider(
            id='rotation-x-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Camera Rotation (Y-axis)'),
        dcc.Slider(
            id='rotation-y-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Camera Rotation (Z-axis)'),
        dcc.Slider(
            id='rotation-z-slider',
            min=-180,
            max=180,
            value=0,
            marks={i: str(i) for i in range(-180, 181, 45)},
            step=1
        ),
        html.Label('Grid Size'),
        dcc.Slider(
            id='grid-size-slider',
            min=2,
            max=100,  # Set maximum grid size to 100
            value=default_grid_size,
            marks={i: str(i) for i in range(2, 101, 10)},
            step=1
        )
    ]),

    # Input for custom formula
    html.Div([
        html.Label('Custom Formula for Surface (e.g., np.sin(np.sqrt(X**2 + Y**2) + time_factor)'),
        dcc.Input(id='formula-input', type='text', value='np.sin(np.sqrt(X**2 + Y**2) + time_factor)', style={'width': '100%'}),
        html.Button('Submit Formula', id='submit-formula-button', n_clicks=0)
    ]),

    # Toggle buttons for surface and grid visibility
    html.Div([
        html.Button('Toggle Surface', id='toggle-surface-button', n_clicks=0),
        html.Button('Toggle Grid', id='toggle-grid-button', n_clicks=0)
    ])
])

# Callback to update the graph
@app.callback(
    Output('3d-surface', 'figure'),
    Input('interval-component', 'n_intervals'),
    Input('current-time-factor', 'data'),
    Input('camera-settings', 'data'),
    Input('zoom-slider', 'value'),
    Input('rotation-x-slider', 'value'),
    Input('rotation-y-slider', 'value'),
    Input('rotation-z-slider', 'value'),
    Input('grid-size-slider', 'value'),
    Input('formula-input', 'value'),
    Input('toggle-surface-button', 'n_clicks'),
    Input('toggle-grid-button', 'n_clicks')
)
def update_graph(n, current_time_factor, camera_settings, zoom, rotation_x, rotation_y, rotation_z, grid_size, formula, surface_clicks, grid_clicks):
    # Generate new mesh and Z values based on the current grid size and formula
    X, Y, Z = generate_mesh_and_z(grid_size, current_time_factor + n * 0.1, formula)

    # Create the figure
    fig = go.Figure()

    # Add mesh surface
    fig.add_trace(go.Surface(
        z=Z,
        x=X,
        y=Y,
        colorscale='Viridis',
        opacity=0.8,
        name='Mesh Surface'
    ))

    # Add B-spline lines
    lines = create_bspline_lines(X, Y, Z)
    for line in lines:
        fig.add_trace(line)

    # Calculate camera position based on rotations
    eye_x = zoom * np.sin(np.radians(rotation_y)) * np.cos(np.radians(rotation_x))
    eye_y = zoom * np.sin(np.radians(rotation_x)) * np.sin(np.radians(rotation_y))
    eye_z = zoom * np.cos(np.radians(rotation_x))

    # Set layout with camera settings
    fig.update_layout(
        title='Nurbs Surface',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            camera=dict(
                eye=dict(x=eye_x, y=eye_y, z=eye_z),  # Updated based on zoom and rotations
                up=dict(x=0, y=0, z=1)
            )
        ),
        showlegend=False
    )

    # Toggle visibility based on button clicks
    if surface_clicks % 2 == 1:
        fig.data[0].visible = not fig.data[0].visible  # Toggle surface visibility

    if grid_clicks % 2 == 1:
        for i in range(1, len(fig.data)):
            fig.data[i].visible = not fig.data[i].visible  # Toggle grid line visibility

    return fig

# Callback to start and stop the animation
@app.callback(
    Output('interval-component', 'disabled'),
    Output('current-time-factor', 'data'),
    Input('start-button', 'n_clicks'),
    Input('stop-button', 'n_clicks'),
    prevent_initial_call=True
)
def toggle_animation(start_clicks, stop_clicks):
    ctx = dash.callback_context

    if ctx.triggered:
        # Determine which button was clicked
        button_id = ctx.triggered[0]['prop_id'].split('.')[0]

        if button_id == 'start-button':
            # Start the animation
            return False, dash.no_update  # Enable the interval

        elif button_id == 'stop-button':
            # Stop the animation
            return True, dash.no_update  # Disable the interval

    return True, dash.no_update  # Default to disabled

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)